### Framework imports

In [ ]:
from noronha.tools.serving import OnlinePredict
from noronha.tools.shortcuts import model_path

### Application imports

In [19]:
import json
import numpy as np
import joblib

from transformers import AutoTokenizer
import torch

### Loading the model

In [ ]:
clf_path = model_path('model.pkl')
clf = joblib.load(clf_path)

### Defining the prediction function

In [ ]:
model_id = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

label_names = ['atis_flight',          
'atis_airfare',
'atis_ground_service',
'atis_airline',
'atis_abbreviation',
'atis_aircraft',
'atis_flight_time',
'atis_quantity',
'atis_airport',
'atis_distance',
'atis_city',
'atis_ground_fare',
'atis_capacity',
'atis_flight_no',
'atis_meal',
'atis_restriction',
'atis_cheapest']

def recognize_intent(input_text):
    # detaching input
    input_text = json.loads(input_text)
    
    # tokenize the input 
    input_ids = torch.tensor(tokenizer.encode(input_text, add_special_tokens=True)).unsqueeze(0)

    # get the prediction
    with torch.no_grad():
        outputs = clf(input_ids)
        predictions = torch.argmax(outputs.logits, dim=1)
    
    return label_names[predictions]


### Creating the prediction service

In [ ]:
OnlinePredict(predict_func=recognize_intent)()